### Types of models to train

Your final submission should include single model. 
The model set you should try to come up with best model per type of model:
1. Identify best model from: Sklearn Logistic Regression - try all combinations of regularization

**Evaluation metric: AUCPR**

### Feature engineering

You should train/fit categorical features scalers and encoders on Train only. Use `transform` or equivalent function on Validation/Test datasets.

It is important to understand all the steps before model training, so that you can reliably replicate and test them to produce scoring function.


You should generate various new features. Examples of such features can be seen in the Module-3 lecture on GLMs.  
Your final model should have at least **10** new engineered features.   
On-hot-encoding, label encoding, and target encoding **is not included in the** **10** features to create.    
You can attempt target encoding, however the technique is not expected to produce improvement for Linear models.


**Note**: 
- It is OK to perform feature engineering using any technique, as long as you can replicate it correctly in the Scoring function.

### Threshold calculation

You will need to calculate optimal threshold for class assignment using F1 metric:
- If using sklearn, use F1 `macro`: `f1_score(y_true, y_pred, average='macro')` 
- If using H2O-3, use F1

You will need to find optimal probability threshold for class assignment, the threshold that maximizes above F1.

In [187]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Loading the CSV file and dropping the index

In [188]:
df = pd.read_csv("D:/Work/Gre/UTD/Courses/Fall/MIS6341/Softwares/Python/ml-fall-2023/Project1/SBA_loans_project_1.csv")
df.drop(columns="index",inplace=True)

# Checking Null Values

In [189]:
df.isnull().sum()

City                   25
State                  12
Zip                     0
Bank                 1405
BankState            1411
NAICS                   0
NoEmp                   0
NewExist              128
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4094
LowDoc               2319
DisbursementGross       0
BalanceGross            0
GrAppv                  0
SBA_Appv                0
MIS_Status              0
dtype: int64

In [190]:
#show unique values in each column and its data type
for col in df.columns:
    print(f'{col} unique values are {df[col].unique()}')
    print("\n")
    print(f'{col} data type is {df[col].dtype}')


City unique values are ['GLEN BURNIE' 'WEST BEND' 'SAN DIEGO' ... 'Orange park' 'GREENHAVEN'
 'SCHAFFERSTOWN']


City data type is object
State unique values are ['MD' 'WI' 'CA' 'MA' 'MO' 'OH' 'IL' 'GA' 'MI' 'NY' 'SC' 'FL' 'KS' 'ID'
 'AZ' 'NH' 'NM' 'KY' 'NJ' 'TX' 'PA' 'MN' 'OK' 'OR' 'WA' 'IN' 'UT' 'AL'
 'MS' 'CO' 'NC' 'CT' 'ME' 'HI' 'LA' 'IA' 'MT' 'RI' 'WV' 'NV' 'AR' 'VA'
 'TN' 'ND' 'VT' 'WY' 'AK' 'SD' 'DE' 'NE' 'DC' nan]


State data type is object
Zip unique values are [21060 53095 92128 ... 32006 56038 14784]


Zip data type is int64
Bank unique values are ['BUSINESS FINANCE GROUP, INC.' 'JPMORGAN CHASE BANK NATL ASSOC'
 'UMPQUA BANK' ... 'WILSHIRE CREDIT CORP' 'NEVADA BANK & TRUST COMPANY'
 'FIRST COMMUN BK OF OZARKS']


Bank data type is object
BankState unique values are ['VA' 'IL' 'OR' 'MA' 'OH' 'CA' 'SD' 'CT' 'RI' 'SC' 'WI' 'GA' 'MI' 'AZ'
 'DE' 'NY' 'NM' 'KY' 'NC' 'NJ' 'MN' 'WA' 'UT' 'IN' 'AL' 'MS' 'TX' 'DC'
 'CO' 'ID' 'PA' 'NH' 'MO' 'MD' 'HI' 'TN' 'IA' 'FL' 'LA' 'MT' nan 'KS' 

# Removing extra variables
In RevLineCr, LowDoc, and NewExist

In [191]:
for i in df['RevLineCr']:
    if i not in ['Y','N']:
        df['RevLineCr'].replace(i,'N',inplace=True)
print("RevLineCr",df['RevLineCr'].unique())

for i in df['LowDoc']:
    if i not in ['Y','N']:
        df['LowDoc'].replace(i,'N',inplace=True)
print("LowDoc",df['LowDoc'].unique())

for i in df['NewExist']:
    if i not in [1,2]:
        df['NewExist'].replace(i,None,inplace=True)
print("NewExist",df['NewExist'].unique())

RevLineCr ['N' 'Y']
LowDoc ['N' 'Y']
NewExist [1.0 2.0 None]


In [192]:
df.isnull().sum()

City                   25
State                  12
Zip                     0
Bank                 1405
BankState            1411
NAICS                   0
NoEmp                   0
NewExist             1060
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr               0
LowDoc                  0
DisbursementGross       0
BalanceGross            0
GrAppv                  0
SBA_Appv                0
MIS_Status              0
dtype: int64

The dataset has a large number of missing values, as can be seen. We'll substitute the correct values for the missing ones. If the column is numeric, for instance, the column median will be used to fill in the missing numbers. In the event that the column is categorical, the mode of the column will be used to fill in the missing values.

# Filling Categorical Values
With the mode aggregation

In [193]:
category_cols=['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc','NewExist']
for column in category_cols:
  df[column]=df[column].fillna(df[column].mode()[0])

In [194]:
df.isnull().sum()

City                 0
State                0
Zip                  0
Bank                 0
BankState            0
NAICS                0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementGross    0
BalanceGross         0
GrAppv               0
SBA_Appv             0
MIS_Status           0
dtype: int64

In [195]:
df.head()
h2o_df = df.copy() # Keeping dataset handy for H2O-GLM Model

After the missing values have been restored, the dataset will be divided into training and testing datasets. Let's divide the dataset into 30% for testing and 70% for training. For repeatability, random state 123 is used.

In [196]:
from sklearn.model_selection import train_test_split

X_train,X_test = train_test_split(df,test_size=0.3,random_state=123)
X_train.shape, X_test.shape

((566472, 19), (242775, 19))

Training set has 566472 rows and testing set has 242775 samples

Target encoding is a data preprocessing technique used to convert categorical variables into numerical values that can be used by machine learning algorithms. It works by replacing each category with the average value of the target variable for that category. This can be helpful for algorithms that cannot handle categorical variables directly.

In this case the target variable is "MIS_Status"

In [197]:
# Target encoder
import category_encoders as ce
categorical_columns = ['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc','NewExist', 'UrbanRural']

encoder = ce.TargetEncoder(cols=categorical_columns)
encoder.fit(X_train, X_train['MIS_Status'])

train_encoded = encoder.transform(X_train)
test_encoded = encoder.transform(X_test)

# Renaming the columns
train_encoded.rename(columns={col: col + "_trg" if col in categorical_columns else col for col in train_encoded.columns}, inplace=False)
test_encoded.rename(columns={col: col + "_trg" if col in categorical_columns else col for col in test_encoded.columns}, inplace=False)

train_encoded.head()

,City,State,Zip,Bank,BankState,NAICS,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
217613,0.114650,0.184773,93001,0.031447,0.218517,235910,56,0.17044,0,0,1,0.071674,0.15307,0.187063,305000.0,0.0,305000.0,244000.0,0
95730,0.137597,0.165992,44039,0.128698,0.159167,484121,0,0.17044,0,0,0,0.243491,0.15307,0.187063,24000.0,0.0,24000.0,12000.0,0
780446,0.139151,0.116799,68122,0.175694,0.159167,451120,20,0.17044,5,15,38510,0.243491,0.15307,0.187063,870400.0,0.0,974500.0,730875.0,0
337263,0.140704,0.197662,14208,0.118949,0.167297,321114,50,0.17044,0,0,1,0.071674,0.15307,0.187063,200000.0,0.0,200000.0,150000.0,0
199634,0.140354,0.144273,16335,0.193277,0.078307,0,19,0.17044,0,0,1,0.071674,0.15307,0.187063,370000.0,0.0,370000.0,296000.0,0


StandardScaler in scikit-learn is a preprocessing technique that centers and scales numerical features such that they have a mean of zero and a standard deviation of one.

We will make use of the StandardScaler, which is used to transform both the training and test data in the same way, ensuring that the features have the same mean and standard deviation in both datasets.

Here we will scale it on the training set and transform on both training and testing

In [198]:
from sklearn.preprocessing import StandardScaler
from copy import deepcopy

numerical_columns = [ 'NoEmp', 'CreateJob', 'RetainedJob', 'GrAppv', 'SBA_Appv', 'DisbursementGross', 'BalanceGross']
scaler = StandardScaler()
train_encoded[numerical_columns] = scaler.fit_transform(train_encoded[numerical_columns])
test_encoded[numerical_columns] = scaler.transform(test_encoded[numerical_columns])

train_encoded.head()



,City,State,Zip,Bank,BankState,NAICS,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
217613,0.114650,0.184773,93001,0.031447,0.218517,235910,0.600407,0.17044,-0.035373,-0.045454,1,0.071674,0.15307,0.187063,0.358949,-0.002296,0.394801,0.410973,0
95730,0.137597,0.165992,44039,0.128698,0.159167,484121,-0.153600,0.17044,-0.035373,-0.045454,0,0.243491,0.15307,0.187063,-0.614207,-0.002296,-0.594552,-0.600302,0
780446,0.139151,0.116799,68122,0.175694,0.159167,451120,0.115688,0.17044,-0.013978,0.018584,38510,0.243491,0.15307,0.187063,2.317036,-0.002296,2.751996,2.533233,0
337263,0.140704,0.197662,14208,0.118949,0.167297,321114,0.519620,0.17044,-0.035373,-0.045454,1,0.071674,0.15307,0.187063,-0.004686,-0.002296,0.025114,0.001232,0
199634,0.140354,0.144273,16335,0.193277,0.078307,0,0.102223,0.17044,-0.035373,-0.045454,1,0.071674,0.15307,0.187063,0.584056,-0.002296,0.623655,0.637638,0


We have created Feature extraction by making use of old variables in the following way


(1) Log_Disbursement which gives the natural logarithmic form of DisbursementGross variable

(2) Log_GrAppv the logarithmic version of the approved loan amount by the bank

(3) Log_SBA_Appv, the logarithmic amount of the approved loan that will be assisted by SBA 

(4) Log_BalanceGross, is the logarithmic amount of total amount in an account or the total value of a financial asset or liability before any deductions or adjustments are made.

(5) TotalJobs variable which is an addition of Createjobs(New people recruited) and RetainedJob (workers working before)

(6) IncomeToLoan its values are calculated by dividing the 'DisbursementGross' column by the 'SBA_Appv' column for each corresponding row. This ratio can help you analyze the relationship between the amount disbursed and the approved SBA loan amount in terms of income.

(7)  EmployeesToLoanRatio, its values are calculated by dividing the 'NoEmp' column (number of employees) by the 'SBA_Appv' column (approved SBA loan amount) for each corresponding row. This ratio can help you analyze the relationship between the number of employees and the size of the SBA loan approved for each entry in the dataset.

(8) JobPerLoan, its values are calculated by dividing the 'TotalJobs' column (representing the total number of jobs) by the 'SBA_Appv' column (approved SBA loan amount) for each corresponding row. This ratio can help you analyze the impact of the SBA loan on job creation or support, expressed as the number of jobs per unit of loan amount approved.

(9) Gauren_SBA_Appv, Its values are calculated by dividing the 'GrAppv' column (gross amount approved by the lender) by the 'SBA_Appv' column (the approved SBA loan amount) for each corresponding row. This ratio helps you analyze the extent to which the SBA is guaranteeing the loan relative to the total loan amount approved by the lender.

(10) DefaultRate, Finally, we create a new feature 'DefaultRate' in the 'train_encoded' DataFrame and set its value to the calculated default rate for the particular group of loans based on the "MIS_Status" variable. This feature will represent the percentage of loans in the group that are classified as defaults.

In [199]:
# Adding Features
import numpy as np
# Apply the log transformation to the specific feature in your training data
#small_constant = 1e-10  # You can adjust this constant as needed
# df['LogColumn'] = np.log(df['OriginalColumn'] + small_constant)
train_encoded['Log_DisbursementGross'] = np.log1p(train_encoded['DisbursementGross'])
train_encoded['Log_GrAppv'] = np.log1p(train_encoded['GrAppv'])
train_encoded['Log_SBA_Appv'] = np.log1p(train_encoded['SBA_Appv'])
train_encoded['Log_BalanceGross'] = np.log1p(train_encoded['BalanceGross'])
train_encoded['TotalJobs'] = train_encoded['CreateJob'] + train_encoded['RetainedJob']
#train_encoded['Loan_Efficiency'] = train_encoded['DisbursementGross'] / (train_encoded['CreateJob'] + train_encoded['RetainedJob'] + 1)
# Calculate 'LoanToIncomeRatio' as a ratio of 'SBA_Appv' to 'DisbursementGross'
train_encoded['IncomeToLoanRatio'] = train_encoded['DisbursementGross'] / train_encoded['SBA_Appv']
# Calculate 'LoanToEmployeesRatio' as a ratio of 'SBA_Appv' to 'NoEmp'
train_encoded['EmployeesToLoanRatio'] = train_encoded['NoEmp'] / train_encoded['SBA_Appv']
# Create a binary feature to indicate loans with a balance ('BalanceGross' > 0)
#train_encoded['HasBalance'] = (train_encoded['BalanceGross'] > 0).astype(int)
# Calculate 'LoanPerJob' as a ratio of 'SBA_Appv' to 'TotalJobs'
train_encoded['JobPerLoan'] = train_encoded['TotalJobs'] / train_encoded['SBA_Appv'] 
# Calculate SBA's Gaurenteed Portion of Approved Loan
train_encoded['Gauren_SBA_Appv'] = train_encoded['GrAppv'] / train_encoded['SBA_Appv']
# Filter the DataFrame to include only the relevant rows
default_group = train_encoded[train_encoded['MIS_Status'].isin([0, 1])]
# Calculate the total number of loans in the filtered group
total_loans = len(default_group)
# Calculate the number of defaults (CHGOFF) in the filtered group
default_loans = len(default_group[default_group['MIS_Status'] == 1])
# Calculate the default rate as a percentage
default_rate = (default_loans / total_loans) * 100
# Create a new feature 'DefaultRate' with the calculated default rate
train_encoded['DefaultRate'] = default_rate


In [200]:
train_encoded.columns

Index(['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'NoEmp',
       'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural',
       'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv',
       'SBA_Appv', 'MIS_Status', 'Log_DisbursementGross', 'Log_GrAppv',
       'Log_SBA_Appv', 'Log_BalanceGross', 'TotalJobs', 'IncomeToLoanRatio',
       'EmployeesToLoanRatio', 'JobPerLoan', 'Gauren_SBA_Appv', 'DefaultRate'],
      dtype='object')

In [201]:
train_encoded.head()

,City,State,Zip,Bank,BankState,NAICS,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status,Log_DisbursementGross,Log_GrAppv,Log_SBA_Appv,Log_BalanceGross,TotalJobs,IncomeToLoanRatio,EmployeesToLoanRatio,JobPerLoan,Gauren_SBA_Appv,DefaultRate
217613,0.114650,0.184773,93001,0.031447,0.218517,235910,0.600407,0.17044,-0.035373,-0.045454,1,0.071674,0.15307,0.187063,0.358949,-0.002296,0.394801,0.410973,0,0.306712,0.332752,0.344279,-0.002298,-0.080828,0.873414,1.460941,-0.196674,0.960651,17.509603
95730,0.137597,0.165992,44039,0.128698,0.159167,484121,-0.153600,0.17044,-0.035373,-0.045454,0,0.243491,0.15307,0.187063,-0.614207,-0.002296,-0.594552,-0.600302,0,-0.952455,-0.902762,-0.917047,-0.002298,-0.080828,1.023163,0.255872,0.134645,0.990421,17.509603
780446,0.139151,0.116799,68122,0.175694,0.159167,451120,0.115688,0.17044,-0.013978,0.018584,38510,0.243491,0.15307,0.187063,2.317036,-0.002296,2.751996,2.533233,0,1.199072,1.322288,1.262213,-0.002298,0.004606,0.914656,0.045668,0.001818,1.086357,17.509603
337263,0.140704,0.197662,14208,0.118949,0.167297,321114,0.519620,0.17044,-0.035373,-0.045454,1,0.071674,0.15307,0.187063,-0.004686,-0.002296,0.025114,0.001232,0,-0.004697,0.024804,0.001231,-0.002298,-0.080828,-3.803563,421.786886,-65.609474,20.385638,17.509603
199634,0.140354,0.144273,16335,0.193277,0.078307,0,0.102223,0.17044,-0.035373,-0.045454,1,0.071674,0.15307,0.187063,0.584056,-0.002296,0.623655,0.637638,0,0.459989,0.484680,0.493255,-0.002298,-0.080828,0.915969,0.160316,-0.126761,0.978071,17.509603


In [202]:
print(train_encoded.head())
h2o_df = train_encoded.copy()
# Saving  as a h2o_df.csv which will be imported directly in h2O for training as all encoding, scaling and feature extraction has already been done during Scikit learn model
h2o_df.to_csv('D:/Work/Gre/UTD/Courses/Fall/MIS6341/Softwares/Python/ml-fall-2023/Project1/h2o_df.csv', index=False)

            City     State    Zip      Bank  BankState   NAICS     NoEmp  \
217613  0.114650  0.184773  93001  0.031447   0.218517  235910  0.600407   
95730   0.137597  0.165992  44039  0.128698   0.159167  484121 -0.153600   
780446  0.139151  0.116799  68122  0.175694   0.159167  451120  0.115688   
337263  0.140704  0.197662  14208  0.118949   0.167297  321114  0.519620   
199634  0.140354  0.144273  16335  0.193277   0.078307       0  0.102223   

        NewExist  CreateJob  RetainedJob  FranchiseCode  UrbanRural  \
217613   0.17044  -0.035373    -0.045454              1    0.071674   
95730    0.17044  -0.035373    -0.045454              0    0.243491   
780446   0.17044  -0.013978     0.018584          38510    0.243491   
337263   0.17044  -0.035373    -0.045454              1    0.071674   
199634   0.17044  -0.035373    -0.045454              1    0.071674   

        RevLineCr    LowDoc  DisbursementGross  BalanceGross    GrAppv  \
217613    0.15307  0.187063           0.35

In [203]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

target_col = "MIS_Status"
cols_to_drop = ['City', 'State', 'Zip','Bank', 'BankState', 'LowDoc','RevLineCr','MIS_Status', 'index']
y = train_encoded[target_col]
X = train_encoded.drop(columns=[target_col])

param_grid = {'C':[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                'penalty': ['l2', 'l1', 'elasticnet'],
                'solver': ['lbfgs'],
                 'l1_ratio': [0.1, 0.3, 0.7] # 'newton-cholesky', 'sag'
                }


clf = LogisticRegression(max_iter=1000, random_state=0, n_jobs=-1)
columns_to_train = [x for x in X.columns if x not in cols_to_drop]
print("Training on following columns:", columns_to_train)
clf.fit(X[columns_to_train], y)
grid1 = GridSearchCV(clf.fit(X[columns_to_train], y), 
                    param_grid, cv =7 , return_train_score= True)
grid1.fit(X[columns_to_train], y)
print("Best parameters found: ", grid1.best_params_)
print("Best cross-validation score: {:.2f}".format(grid1.best_score_))

Training on following columns: ['NAICS', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv', 'Log_DisbursementGross', 'Log_GrAppv', 'Log_SBA_Appv', 'Log_BalanceGross', 'TotalJobs', 'IncomeToLoanRatio', 'EmployeesToLoanRatio', 'JobPerLoan', 'Gauren_SBA_Appv', 'DefaultRate']
Best parameters found:  {'C': 10, 'l1_ratio': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best cross-validation score: 0.82


Best Hyperparameters Found:

    'C': 10  
    In logistic regression, the parameter 'C' is the regularization strength or the inverse of the regularization parameter. It controls the trade-off between fitting the training data well and preventing overfitting.
A smaller 'C' value increases the regularization strength, leading to simpler models with smaller coefficients. This helps to prevent overfitting but may sacrifice some accuracy on the training data.
A larger 'C' value decreases the regularization strength, allowing the model to fit the training data more closely. However, this may lead to overfitting.


    'penalty': 'l2'
        L2 regularization is a type of regularization used in logistic regression. It adds a penalty term to the loss function, which encourages the model to keep all feature weights small.
    The penalty term is proportional to the square of the magnitude of feature coefficients. It helps prevent overfitting by reducing the influence of individual features in the model.
    L2 regularization is controlled by the 'C' parameter, with a smaller 'C' increasing the strength of regularization and leading to smaller feature coefficients.
    'solver': 'lbfgs'
    LBFGS is an optimization algorithm used for logistic regression and other machine learning models.
It's an iterative numerical optimization algorithm that finds the optimal values of the model parameters (coefficients) by minimizing the logistic regression loss function.
LBFGS is suitable for problems with a large number of features, and it efficiently approximates the Hessian matrix, which is used to update the model parameters.
It is one of the solvers available for logistic regression in scikit-learn and is known for its efficiency and effectiveness.

Best Cross-Validation Score:

    The best cross-validation score achieved by the model is 0.82.

In [204]:
import pickle

# Assuming you have a variable named 'best_params' containing the best hyperparameters
best_params = grid1.best_params_

with open('best_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)

In [205]:
from sklearn.metrics import f1_score
import numpy as np

def calculate_optimal_threshold(classifier, X, y):
    # Predict probabilities
    y_prob = classifier.predict_proba(X)[:, 1]
    
    # Generate a range of thresholds
    thresholds = np.linspace(0, 1, 100)
    
    # Compute F1 scores for different thresholds
    f1_scores = [f1_score(y, (y_prob > threshold).astype(int), average='macro') for threshold in thresholds]
    
    # Find the optimal threshold with the highest F1 score
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    
    return optimal_threshold